# Import packages

In [3]:
import pandas as pd
from pulp import *
from random import randint

# Import data
## Defining which dataset you want to use

In [286]:
pred = int(input('Which predictions do you want to use? Press 1 for consultancy and 0 for our own predictions '))

## Generating the dictionary containing the engines and their respective RULs

In [287]:
if pred == 1:
    RUL_data = pd.read_excel('RUL_consultancy_predictions.xlsx')
else:
    RUL_data = pd.read_excel('RUL_predictions.xlsx')

T = 25 #planning horizon

# rounding the predicted RUL values
RUL_data.RUL = RUL_data['RUL'].round()

# store the RUL values (as integers) in a list 
RUL_list = list(RUL_data.RUL.astype(int))

# dictionary with engine numbers as keys and RUL values
RUL_raw = {engine: RUL_list[engine-1] for engine in range(1,101)}

RUL = {engine: value for engine, value in RUL_raw.items() if value < T}

## Importing information about teams, times and precalculating the costs

In [288]:
time_for_A = {}
for engine in range(1,101):
    if engine <= 25:
        time_for_A[engine] = 4
    elif engine <= 50:
        time_for_A[engine] = 6
    elif engine <= 75:
        time_for_A[engine] = 3
    else:
        time_for_A[engine] = 5

In [289]:
time_for_B = {}
for engine in range(1,101):
    if engine <= 33:
        time_for_B[engine] = time_for_A[engine] + 1
    elif engine <= 67:
        time_for_B[engine] = time_for_A[engine] + 2
    else:
        time_for_B[engine] = time_for_A[engine] + 1

In [290]:
cost = {}
for engine in range(1,101):
    if engine <= 20:
        cost[engine] = 5
    elif engine <= 40:
        cost[engine] = 7
    elif engine <= 60:
        cost[engine] = 9
    elif engine <= 80:
        cost[engine] = 5
    else:
        cost[engine] = 3

In [291]:
engines = list(RUL.keys()) #list of engines
teams = ['A','A','B','B'] # 2teams of type A, 2teams of type B
nr_teams = list(range(len(teams))) #list of teams with unique team number
times = range(1,T+1)
times_inclusive = range(0,T+1)

# create a dictionary with unique teams (keys) having a dictionary (value) of maintenance 
# time required for each engine (considering the team type)
maintenance_time = {}
for team in range(len(teams)):
    if teams[team] == 'A':
        maintenance_time[team] = time_for_A
    else:
        maintenance_time[team] = time_for_B

In [292]:
max_engines = {}
for team in nr_teams:
    if team == 0 or team == 1:
        max_engines[team] = 2
    else:
        max_engines[team] = 2

In [293]:
penalty = {}
for q in nr_teams:
    penalty[q] = {j: {t: max(0, min(cost[j]*(T-RUL[j]), cost[j]*(t+maintenance_time[q][j]-1-RUL[j]))) for t in times_inclusive} for j in engines}
    
for q in nr_teams:
    for j in engines:
        penalty[q][j][0] = penalty[q][j][T]
        

In [294]:
penalty_total = {}
for j in engines:
    penalty_total[j] = (T - RUL[j])*cost[j]

# Create the model

In [295]:
# Create a minimization model
model = LpProblem("Airplane_Maintenance_Problem", LpMinimize)

In [296]:
#Declare Variables

# Binary decision variable to see if team 'q' starts working on engine 'j' at time t 
x = LpVariable.dicts('x',indexs = (nr_teams, engines, times), lowBound=0, upBound=1, cat='Binary')

In [297]:
# Objective function
model += lpSum(lpSum(lpSum(x[q][j][t]*penalty[q][j][t] for t in times) for j in engines) for q in nr_teams) + \
lpSum((1-lpSum(lpSum((x[q][j][t]) for t in times) for q in nr_teams))*penalty_total[j] for j in engines)

In [298]:
# No runover the end of T
for j in engines:
    model += lpSum(lpSum(x[q][j][t]*(t + maintenance_time[q][j]-1) for t in times) for q in nr_teams) <= T

In [299]:
 # Constraint that only 1 team can work on an engine
for j in engines:
    model += lpSum(lpSum(x[q][j][t] for q in nr_teams) for t in times) <= 1

In [300]:
# Constraint that work cannot start on two engines at the same time
for t in times:
    for q in nr_teams:
        model += lpSum(x[q][j][t] for j in engines) <= 1

In [301]:
# Constraint that the team can only work on 2 engines if T=25

for q in nr_teams:
    model += lpSum(lpSum(x[q][j][t] for j in engines) for t in times) <= max_engines[q]

In [302]:
K = 69

# Overlap constraint
for q in nr_teams:
    for t0 in times:
        for j1 in engines:
            model += K*x[q][j1][t0]+(lpSum(lpSum(x[q][j2][t] for j2 in engines) for t in range(t0+1,min(t0+maintenance_time[q][j1]-1,T)+1))) <= K
            

# Solve the model

In [278]:
model.solve()

#The status of the solution is printed on the screen 
print("Status = %s" % LpStatus[model.status])

# The optimised objective function value is printed to the screen
print( "Minimum Cost = " +str( value(model.objective) ) )

Status = Optimal
Minimum Cost = 227.0


# Output to excel files

In [279]:
output = {}
for q in nr_teams:
    for t in times:
        for j in engines:
            if x[q][j][t].varValue == 1:
                output.setdefault(q, {})[j] = {'start time': t, 'finish time': t+maintenance_time[q][j]-1, 'maintenance_time': maintenance_time[q][j]}

In [280]:
output_df = pd.DataFrame.from_dict({(i,j): output[i][j] 
                           for i in output.keys() 
                           for j in output[i].keys()},
                       orient='index')

output_df = output_df.rename_axis(['team', 'engine'])
output_df = output_df.sort_values(['team', 'start time'])

In [282]:
costs_per_engine = {}

for t in times:
    for j in engines:
        for q in nr_teams:
            if x[q][j][t].varValue == 1:
                costs_per_engine[j] = penalty[q][j][t]

for j in engines:
    if j not in costs_per_engine.keys():
        costs_per_engine[j] = penalty_total[j]

In [283]:
costs_per_engine_df = pd.DataFrame.from_dict(costs_per_engine, orient = 'index', columns = ['costs'])
costs_per_engine_df = costs_per_engine_df.rename_axis('engine')
costs_per_engine_df = costs_per_engine_df.sort_values(['engine'])
costs_per_engine_df['total costs'] = costs_per_engine_df['costs'].cumsum()

In [ ]:
if pred==1:
    with pd.ExcelWriter('Optimization2C.xls') as writer:  # doctest: +SKIP
         output_df.to_excel(writer, sheet_name='schedule')
         costs_per_engine_df.to_excel(writer, sheet_name='costs')
else:
    with pd.ExcelWriter('Optimization2B.xls') as writer:  # doctest: +SKIP
         output_df.to_excel(writer, sheet_name='schedule')
         costs_per_engine_df.to_excel(writer, sheet_name='costs')

# Check the results

In [208]:
cost = 0
for q in nr_teams:
    for t in times:
        for j in engines:
            if x[q][j][t].varValue == 1:
                print("At time %i, team %i starts work on engine: %i, penalty = %i, maintenance time = %i, finish time = %i, RUL = %i" % (t, q, j, penalty[q][j][t], 
                                                                                                              maintenance_time[q][j], t + maintenance_time[q][j] - 1, RUL[j]))
print("Total costs =", value(model.objective))                

At time 1, team 0 starts work on engine: 34, penalty = 0, maintenance time = 6, finish time = 6, RUL = 6
At time 7, team 0 starts work on engine: 49, penalty = 0, maintenance time = 6, finish time = 12, RUL = 14
At time 1, team 1 starts work on engine: 76, penalty = 0, maintenance time = 5, finish time = 5, RUL = 6
At time 6, team 1 starts work on engine: 35, penalty = 0, maintenance time = 6, finish time = 11, RUL = 11
At time 1, team 2 starts work on engine: 68, penalty = 0, maintenance time = 4, finish time = 4, RUL = 9
At time 5, team 2 starts work on engine: 31, penalty = 0, maintenance time = 7, finish time = 11, RUL = 11
At time 4, team 3 starts work on engine: 42, penalty = 0, maintenance time = 8, finish time = 11, RUL = 11
At time 12, team 3 starts work on engine: 56, penalty = 0, maintenance time = 5, finish time = 16, RUL = 16
Total costs = 227.0
